In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
#from sentence_embeddings import SentenceEmbeddings

trained_model_path = '/content/drive/MyDrive/t5v1_tcc_2024/model'
trained_tokenizer_path = '/content/drive/MyDrive/t5v1_tcc_2024/tokenizer'


class QuestionGeneration:

    def __init__(self, model_dir=None):
        self.model = T5ForConditionalGeneration.from_pretrained(trained_model_path)
        self.tokenizer = T5Tokenizer.from_pretrained(trained_tokenizer_path)
        self.device = torch.device("cuda")
        self.model = self.model.to(self.device)
        self.model.eval()

    def generate(self, answer: str, context: str):
        input_text = '<answer> %s <context> %s ' % (answer, context)
        encoding = self.tokenizer.encode_plus(
            input_text,
            return_tensors='pt'
        ).to(self.device)
        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']
        outputs = self.model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            num_beams = 3,
            num_return_sequences = 1,
            max_length=512
        ).to(self.device)
        question_list = []
        for output in outputs:
            question = self.tokenizer.decode(
                output,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True
            )
            question_list.append({'question': question, 'answer': answer, 'context': context})
        return question_list



context = '''
O T5 é baseado na arquitetura Transformer, que utiliza mecanismos de autoatenção para capturar relacionamentos de longo alcance em sequências de texto. Isso permite que o modelo aprenda representações de alta qualidade para uma ampla variedade de tarefas de processamento de linguagem natural (PLN).
'''
answer_list = ['T5', 'texto']

QG = QuestionGeneration()
# #SE = SentenceEmbeddings()

for answer in answer_list:
    #qa_pair_list = QG.generate(answer, context)
    #most_similar = SE.get_most_similar(context, qa_pair_list)
    #print(most_similar)
    qa = QG.generate(answer, context)
    print(qa)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[{'question': 'Qual modelo é baseado em a arquitetura Transformer?', 'answer': 'T5', 'context': '\nO T5 é baseado na arquitetura Transformer, que utiliza mecanismos de autoatenção para capturar relacionamentos de longo alcance em sequências de texto. Isso permite que o modelo aprenda representações de alta qualidade para uma ampla variedade de tarefas de processamento de linguagem natural (PLN).\n'}]
[{'question': 'Que tipo de sequências o T5 captura relacionamentos de longo alcance?', 'answer': 'texto', 'context': '\nO T5 é baseado na arquitetura Transformer, que utiliza mecanismos de autoatenção para capturar relacionamentos de longo alcance em sequências de texto. Isso permite que o modelo aprenda representações de alta qualidade para uma ampla variedade de tarefas de processamento de linguagem natural (PLN).\n'}]


In [ ]:
!pip install datasets evaluate transformers[sentencepiece]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [ ]:
import nltk
import editdistance
from datasets import load_metric

teste 2

In [ ]:
!pip install Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 88.8 MB/s eta 0:00:00


In [ ]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=6e95c25ec80cecb8662305dcb00251d7486e3ddaa58f6ee51e6fc3b421d857cc
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
import Levenshtein
from rouge_score import rouge_scorer

tokenizer = QG.tokenizer
model = QG.model

# Carregar o conjunto de dados SQuAD
dataset = load_dataset("squad_v1_pt")
dataset = dataset.shuffle()
small_validation_dataset = dataset["validation"].select(range(5000))
# Processar o conjunto de dados para obter pares de perguntas e respostas de referência
references = []
for example in small_validation_dataset:
    references.append(example["question"])

# Gerar respostas para as perguntas usando o modelo
generated_responses = []
for example in small_validation_dataset:
    input_text = "question: {} context: {}".format(example["question"], example["context"])
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512,truncation=True)
    output = model.generate(input_ids.to(model.device))
    generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
    generated_responses.append(generated_response)

# Calcular BLEU
bleu_scores = corpus_bleu(references, generated_responses, smoothing_function=SmoothingFunction().method1)

# Inicialize o objeto scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

# Calcule o ROUGE
for index in range(len(small_validation_dataset)):
    print(references[index])
    print(generated_responses[index])
    scores = scorer.score(references[index], generated_responses[index])

# Acessar os scores
rouge_1_score = scores['rouge1'].fmeasure
rouge_l_score = scores['rougeL'].fmeasure

print("ROUGE-1 Score:", rouge_1_score)
print("ROUGE-L Score:", rouge_l_score)

# Calcular Levenshtein Distance
levenshtein_distances = [Levenshtein.distance(ref, gen) for ref, gen in zip(references, generated_responses)]
average_levenshtein_distance = sum(levenshtein_distances) / len(levenshtein_distances)

# Imprimir os resultados
print("BLEU Score:", bleu_scores)
print("Average Levenshtein Distance:", average_levenshtein_distance)


A saída de streaming foi truncada nas últimas 5000 linhas.
Contra o que o oxigênio reativo desempenha na defesa das plantas?
Qual é o on superóxido?
Onde fica uma casa de palmeiras com plantas subtropicais de todo o mundo?
Qual é o nome da casa de palmeira na Nova Oranger
Quantas áreas metropolitanas estendidas existem?
Qual é a área estatstica combinada na
Quem pegou um passe de 16 jardas nesta unidade?
Qual foi o ttulo da recepço de Devin
Em que ano a ABC lançou o Filme da Semana?
Qual foi o nome do programa de filmes feito
Quem foi o receptor da conversão bem-sucedida de 2 pontos?
Qual foi o objetivo da converso de 2 pontos?
Depois de rejeitar a confissão compulsória, o que Lutero pediu?
Qual foi o ensaio sobre a Confiss
O que ele achou que poderia constituir um perigo?
Qual foi o relatório de Trump?
Em que escola Hank Marvin e Bruce Welch participaram?
Quais so os dois exemplos de alun
Onde Martin Luther foi para a escola?
Qual foi o nome da escola que Martin Luther ingressou?
Ness

teste 3

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
import Levenshtein
from rouge_score import rouge_scorer

tokenizer = QG.tokenizer
model = QG.model

# Carregar o conjunto de dados SQuAD
dataset = load_dataset("squad_v1_pt")
# Processar o conjunto de dados para obter pares de perguntas e respostas de referência
references = []
for example in dataset["validation"]:
    references.append(example["question"])

# Gerar respostas para as perguntas usando o modelo
generated_responses = []
for example in dataset["validation"]:
    input_text = "question: {} context: {}".format(example["question"], example["context"])
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512,truncation=True)
    output = model.generate(input_ids.to(model.device))
    generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
    generated_responses.append(generated_response)

# Calcular BLEU
bleu_scores = corpus_bleu(references, generated_responses, smoothing_function=SmoothingFunction().method1)

# Inicialize o objeto scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

# Calcule o ROUGE
for index in range(len(small_validation_dataset)):
    print(references[index])
    print(generated_responses[index])
    scores = scorer.score(references[index], generated_responses[index])

# Acessar os scores
rouge_1_score = scores['rouge1'].fmeasure
rouge_l_score = scores['rougeL'].fmeasure

print("ROUGE-1 Score:", rouge_1_score)
print("ROUGE-L Score:", rouge_l_score)

# Calcular Levenshtein Distance
levenshtein_distances = [Levenshtein.distance(ref, gen) for ref, gen in zip(references, generated_responses)]
average_levenshtein_distance = sum(levenshtein_distances) / len(levenshtein_distances)

# Imprimir os resultados
print("BLEU Score:", bleu_scores)
print("Average Levenshtein Distance:", average_levenshtein_distance)

A saída de streaming foi truncada nas últimas 5000 linhas.
Lutero tratou os judeus da mesma maneira que tratou que grupo?
Qual foi a razo pela qual Lutero tratou
Quando Lutero escreveu um tratado extremo contra os judeus?
Qual foi o ltimo tratado de Lutero contra
Como Lutero viu os turcos otomanos?
Qual foi o objetivo de Lutero em travar
Qual foi o propósito dos turcos na mente de Lutero?
Qual foi o propósito dos turcos na
Já que Lutero acreditava que os turcos foram enviados por Deus, qual era o seu propósito?
Qual foi o propósito de Lutero a envia
Quem Lutero viu para ser o Anticristo?
Qual foi o objetivo de Lutero em seu
Que tipo de guerra Lutero apoiou contra os turcos, mesmo que ele não se opusesse a uma guerra religiosa?
Qual foi o objetivo de Lutero na guerra
Que livro Lutero leu em 1542?
Qual é o nome do livro que Lutero leu e
Que tipo de escritos a leitura do Alcorão trouxe em Lutero?
Qual é o escrito que Lutero acreditava
O que era o maometismo e o turco?
Quais foram os dois 

teste 4


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
import Levenshtein
from rouge_score import rouge_scorer

tokenizer = QG.tokenizer
model = QG.model

# Carregar o conjunto de dados SQuAD
dataset = load_dataset("squad_v1_pt")
dataset = dataset.shuffle()
small_validation_dataset = dataset["validation"].select(range(5000))
# Processar o conjunto de dados para obter pares de perguntas e respostas de referência
references = []
for example in small_validation_dataset:
    print(example["answers"]["text"])
    references.append(example["question"])

# Gerar respostas para as perguntas usando o modelo
generated_responses = []
for example in small_validation_dataset:
    input_text = "answer: {} context: {}".format(example["answers"]["text"][0], example["context"])
    print(input_text)
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512,truncation=True)
    output = model.generate(input_ids.to(model.device))
    generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
    generated_responses.append(generated_response)

# Calcular BLEU
bleu_scores = corpus_bleu(references, generated_responses, smoothing_function=SmoothingFunction().method1)

# Inicialize o objeto scorer
scorer = rouge_scorer.RougeScorer(['rougen', 'rougeL'], use_stemmer=True)

# Calcule o ROUGE
for index in range(len(small_validation_dataset)):
    print(references[index])
    print(generated_responses[index])
    scores = scorer.score(references[index], generated_responses[index])

# Acessar os scores
rouge_n_score = scores['rougen'].
rouge_l_score = scores['rougeL'].fmeasure

print("ROUGE-1 Score:", rouge_1_score)
print("ROUGE-L Score:", rouge_l_score)

# Calcular Levenshtein Distance
levenshtein_distances = [Levenshtein.distance(ref, gen) for ref, gen in zip(references, generated_responses)]
average_levenshtein_distance = sum(levenshtein_distances) / len(levenshtein_distances)

# Imprimir os resultados
print("BLEU Score:", bleu_scores)
print("Average Levenshtein Distance:", average_levenshtein_distance)

A saída de streaming foi truncada nas últimas 5000 linhas.
O que os membros não eleitos do governo escocês podem fazer?
Qual é o exemplo de um hemiciclo?
De que duração são os eventos do ciclo do motor quando são usadas as engrenagens de válvulas mais simples?
Qual é o comprimento fixo durante o cicl
O que causa tensão nas estruturas?
Qual é o termo para a área transversal relevante
Quantas vezes o Arizona virou a bola no NFC Championship?
Qual foi o nmero de jardas e forçand
Para o que pode o uso de respiração prolongada de oxigênio a 60 kPa levar?
Qual é o resultado permanente de respiraço de
De 1947 a 1967, quanto aumentou o preço do petróleo?
Qual foi o preço do petróleo em dó
Embora consts do sul da califórnia de um ambiente urbano altamente desenvolvido, quanto dele foi deixado subdesenvolvido?
Qual é a razo pela qual o sul
Como os socialistas acham que os meios de produção devem ser propriedade?
Qual é o exemplo de propriedade social?
O que a escala de CPI mede?
Qual é o ndice d

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
import Levenshtein
from rouge_score import rouge_scorer

tokenizer = QG.tokenizer
model = QG.model

# Carregar o conjunto de dados SQuAD
dataset = load_dataset("squad_v1_pt")
dataset = dataset.shuffle()
small_validation_dataset = dataset["validation"].select(range(5000))
# Processar o conjunto de dados para obter pares de perguntas e respostas de referência
references = []
for example in small_validation_dataset:
    references.append(example["question"])

# Gerar respostas para as perguntas usando o modelo
generated_responses = []
for example in small_validation_dataset:
    input_text = "question: {} context: {}".format(example["question"], example["context"])
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512,truncation=True)
    output = model.generate(input_ids.to(model.device))
    generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
    generated_responses.append(generated_response)

# Calcular BLEU
bleu_scores = corpus_bleu(references, generated_responses, smoothing_function=SmoothingFunction().method1)

# Inicialize o objeto scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

# Calcule o ROUGE
for index in range(len(small_validation_dataset)):
    print(references[index])
    print(generated_responses[index])
    scores = scorer.score(references[index], generated_responses[index])

# Acessar os scores
rouge_n_score = scores['rouge1'].fmeasure
rouge_l_score = scores['rougeL'].fmeasure

print("ROUGE-n1 Score:", rouge_n_score)
print("ROUGE-L Score:", rouge_l_score)

# Calcular Levenshtein Distance
levenshtein_distances = [Levenshtein.distance(ref, gen) for ref, gen in zip(references, generated_responses)]
average_levenshtein_distance = sum(levenshtein_distances) / len(levenshtein_distances)

# Imprimir os resultados
print("BLEU Score:", bleu_scores)
print("Average Levenshtein Distance:", average_levenshtein_distance)